# Neural Network - MNIST database

In [2]:
import os
import numpy as np
from datetime import datetime
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from keras.datasets import mnist
tf.__version__

Using TensorFlow backend.


'2.0.0'

In [3]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, MaxPool2D, Conv2D,Reshape

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import warnings
warnings.filterwarnings('ignore')
from keras.utils import np_utils

np.random.seed(123) 

In [6]:
# para solucionar problema de que no me cargaba la base de datos MNIST
#import requests
#requests.packages.urllib3.disable_warnings()
#import ssl
#ssl._create_default_https_context = _create_unverified_https_context


In [7]:
#cargar Fashion MNIST
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [8]:
#Normalizar X
max_val = X_train.max()
X_train = X_train / max_val
X_test = X_test / max_val

X_cv, X_test = X_test[7000:], X_test[:7000]
y_cv, y_test = y_test[7000:], y_test[:7000]

In [9]:
logdir = os.path.join("logs", datetime.now().strftime("%Y%m%d-%H%M%S"))
logdir = os.path.join("logs", "run_5")

tensorflow_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stopping_callback = tf.keras.callbacks.EarlyStopping(patience=2)

In [10]:
def my_nn(activation="relu", optimizer="sgd", layers=(300, 100)):
  Xin = Input(shape=(28, 28))
  X = Flatten()(Xin)
  for layer in layers:
    X = Dense(layer, activation=activation)(X)
  
  x = Dropout(0.2)
  X = Dense(10, activation="softmax")(X)

  model = Model(inputs=Xin, outputs=X)

  model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
  
  print(model.summary())  
  return model

In [11]:
tf.keras.backend.clear_session()
model = my_nn(layers=(300, 200, 200,100, 50), activation = 'relu', optimizer='Adamax')
model.fit(x=X_train, y=y_train,
          epochs=100,
          validation_data=(X_cv, y_cv),
           callbacks=[tensorflow_callback, stopping_callback])


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
dense_1 (Dense)              (None, 200)               60200     
_________________________________________________________________
dense_2 (Dense)              (None, 200)               40200     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               20100     
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050  

In [12]:
m_test_loss, m_test_accuracy = model.evaluate(X_test, y_test)


7000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [13]:
print(f"Test accuracy: {m_test_accuracy}")

Test accuracy: 0.8781428337097168


In [1]:
y_predict=np.argmax(model.predict(X_test), axis=-1)
confusion_mat = tf.math.confusion_matrix(labels=y_test, predictions=y_predict)
confusion_mat

NameError: name 'np' is not defined

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(confusion_mat,  
            cmap="YlGnBu",
            annot=True, 
            fmt='g')
plt.ylabel('True')
plt.xlabel('Predicted')
plt.show()